In [0]:
# Databricks notebook source
%pip install faker

In [0]:
# COMMAND ----------
dbutils.library.restartPython()


In [0]:
# COMMAND ----------
from __future__ import annotations

import json
import random
import uuid
from dataclasses import dataclass
from datetime import datetime, timedelta, date
from typing import Tuple

import numpy as np
from faker import Faker


In [0]:
# COMMAND ----------
dbutils.widgets.text("faculdade", "USP ESALQ")
dbutils.widgets.text("qtd_inscritos", "12000")

dbutils.widgets.dropdown(
    "rede_social",
    "tiktok",
    ["tiktok", "instagram_reels", "youtube_shorts"]
)

dbutils.widgets.dropdown(
    "tipo_data",
    "today",
    ["today", "custom"]
)

dbutils.widgets.text(
    "data_custom",
    "2026-01-01"
)

dbutils.widgets.text("seed", "42")

dbutils.widgets.text(
    "base_path",
    "s3://capstone-impacta/Capstone/bronze"
)


In [0]:
# COMMAND ----------
faculdade = dbutils.widgets.get("faculdade").strip()
subs = int(dbutils.widgets.get("qtd_inscritos"))
platform = dbutils.widgets.get("rede_social").strip()

tipo_data = dbutils.widgets.get("tipo_data")
data_custom = dbutils.widgets.get("data_custom")

seed = int(dbutils.widgets.get("seed"))
base_path = dbutils.widgets.get("base_path").rstrip("/")

run_id = str(uuid.uuid4())

if tipo_data == "today":
    day = date.today()
else:
    day = datetime.strptime(data_custom, "%Y-%m-%d").date()


In [0]:
# COMMAND ----------
VIDEO_TITLES_POOL = [

# 🎓 Vida acadêmica em tecnologia
"Como é a rotina de um aluno de tecnologia",
"Vale a pena fazer faculdade de tecnologia hoje?",
"O que ninguém te conta sobre a faculdade de TI",
"Faculdade de tecnologia é difícil mesmo?",
"O maior erro de quem começa tecnologia",
"Tecnologia não é só programação",
"O choque do primeiro semestre em TI",
"Como sobreviver aos primeiros períodos",
"Por que tanta gente desiste de tecnologia",
"O que eu faria diferente se começasse hoje",

# 💻 Programação & Desenvolvimento
"Programar não é decorar código",
"Lógica de programação explicada simples",
"A linguagem certa pra começar em tecnologia",
"Por que aprender fundamentos antes de frameworks",
"O erro clássico de iniciantes em programação",
"Backend ou frontend: qual escolher?",
"Full stack existe mesmo?",
"Como evoluir mais rápido programando",
"O que estudar fora da sala de aula",
"Como montar projetos sem experiência",

# 📚 Estudos & Aprendizado
"Como estudar tecnologia sem surtar",
"O jeito certo de estudar programação",
"Como aprender tecnologia mais rápido",
"Por que copiar código não te ensina",
"Estudar pouco e aprender mais",
"Técnica de estudo que funciona pra TI",
"Como não esquecer o que você estudou",
"O erro ao estudar para provas técnicas",
"Como conciliar estudo e trabalho",
"O que realmente vale estudar na faculdade",

# 🧠 Carreira & Mercado
"Faculdade ainda importa no mercado de tecnologia?",
"O que o mercado espera de um aluno de TI",
"Como conseguir estágio em tecnologia",
"Estágio ou projeto pessoal: o que pesa mais?",
"Como montar currículo sendo estudante",
"O que empresas olham em alunos de tecnologia",
"O maior mito sobre trabalhar com TI",
"Por que diploma sozinho não garante vaga",
"O que diferencia alunos medianos dos bons",
"Como sair da faculdade empregado",

# 🧪 Dados, Cloud & Infra
"Todo aluno de TI deveria aprender dados",
"Cloud não é só pra sênior",
"O básico de Big Data que ninguém ensina",
"Por que SQL é obrigatório em tecnologia",
"Dados é o novo arroz com feijão",
"O que é cloud computing de verdade",
"Por que entender infraestrutura faz diferença",
"Dev que entende dados sai na frente",
"O mínimo de cloud que você precisa saber",
"Como dados e TI se conectam",

# 🔐 Segurança & Boas práticas
"Segurança começa no código",
"O erro de segurança mais comum em projetos",
"Por que segurança não é opcional",
"O básico de LGPD que todo aluno deveria saber",
"Como evitar falhas simples em sistemas",
"Segurança não é só pra hackers",
"Boas práticas que evitam dor de cabeça",
"O que é segurança da informação na prática",
"Erro de iniciante que vira vulnerabilidade",
"Como pensar em segurança desde o início",

# 🧩 Projetos & Portfólio
"Projetos valem mais que notas?",
"Como criar portfólio na faculdade",
"Projeto simples que impressiona recrutador",
"Portfólio ruim pode te eliminar",
"O que NÃO colocar no portfólio",
"Como transformar trabalho da faculdade em projeto real",
"GitHub que chama atenção",
"Como documentar projetos direito",
"Projeto acadêmico ou projeto real?",
"Como mostrar evolução no portfólio",

# 🤖 IA, Automação & Futuro
"IA vai substituir quem trabalha com TI?",
"Como alunos podem usar IA sem trapacear",
"O que estudar hoje pensando no futuro",
"Automação não vai acabar com seu emprego",
"IA como ferramenta de aprendizado",
"O erro ao depender demais de IA",
"Como usar IA pra estudar tecnologia",
"O que muda na carreira com IA",
"Por que entender IA é diferencial",
"IA não substitui base fraca",

# 🎯 Soft skills em tecnologia
"Programador que não sabe comunicar sofre",
"Soft skills importam mais do que você imagina",
"Como explicar código pra quem não é técnico",
"O erro de comunicação em times de TI",
"Trabalhar em grupo em tecnologia",
"Por que bom aluno não vira bom profissional",
"Como receber feedback técnico",
"O que faculdade não ensina sobre trabalho em TI",
"Como se posicionar em entrevistas",
"Postura profissional desde a faculdade",

# 🚀 Produtividade & Organização
"Como organizar estudos em tecnologia",
"Ferramentas que todo aluno de TI deveria usar",
"Como planejar semana de estudos",
"Estudar TI sem procrastinar",
"O erro que rouba seu tempo",
"Como estudar programação todo dia",
"Gestão de tempo para alunos de tecnologia",
"Como não acumular matéria técnica",
"O método mais simples pra estudar TI",
"Rotina produtiva pra estudantes de tecnologia",

# 🔍 Mitos & Verdades
"Precisa ser bom em matemática pra TI?",
"Programar é pra poucos?",
"Faculdade pública ou privada faz diferença?",
"Tecnologia é só pra gênios?",
"Você precisa saber inglês fluente?",
"Dá pra aprender TI sozinho?",
"Certificados substituem faculdade?",
"Idade importa pra começar em tecnologia?",
"TI é só ficar no computador?",
"Quem não programa está fora da área?",

]


CATEGORIES = [
    "Dicas rápidas",
    "Tutorial",
    "Curiosidades",
    "Mitos vs Verdades",
    "Checklist",
    "Erro comum",
    "Top 5",
    "Ferramenta do dia",
    "Case real",
]


In [0]:
# COMMAND ----------
@dataclass
class Baseline:
    view_factor_range: Tuple[float, float]
    like_rate_range: Tuple[float, float]
    comment_rate_range: Tuple[float, float]

def tier(subs: int) -> str:
    if subs < 1_000: return "nano"
    if subs < 10_000: return "micro"
    if subs < 100_000: return "mid"
    if subs < 1_000_000: return "macro"
    return "mega"

def baseline_for(platform: str, subs: int) -> Baseline:
    t = tier(subs)

    if platform == "tiktok":
        view = {"nano":(0.6,3.2),"micro":(0.4,2.4),"mid":(0.25,1.8),"macro":(0.18,1.3),"mega":(0.12,1.0)}
        like = {"nano":(0.04,0.10),"micro":(0.035,0.085),"mid":(0.03,0.07),"macro":(0.025,0.06),"mega":(0.02,0.05)}
        comm = {"nano":(0.003,0.010),"micro":(0.0025,0.008),"mid":(0.002,0.006),"macro":(0.0015,0.0045),"mega":(0.0012,0.004)}
    elif platform == "instagram_reels":
        view = {"nano":(0.5,2.2),"micro":(0.35,1.7),"mid":(0.22,1.25),"macro":(0.16,0.95),"mega":(0.12,0.8)}
        like = {"nano":(0.03,0.08),"micro":(0.028,0.07),"mid":(0.025,0.06),"macro":(0.02,0.05),"mega":(0.018,0.045)}
        comm = {"nano":(0.0025,0.008),"micro":(0.002,0.006),"mid":(0.0016,0.0048),"macro":(0.0012,0.004),"mega":(0.001,0.0035)}
    else:
        view = {"nano":(0.4,2.6),"micro":(0.28,2.0),"mid":(0.20,1.5),"macro":(0.15,1.15),"mega":(0.12,1.0)}
        like = {"nano":(0.035,0.09),"micro":(0.03,0.08),"mid":(0.028,0.07),"macro":(0.024,0.06),"mega":(0.022,0.055)}
        comm = {"nano":(0.002,0.008),"micro":(0.0018,0.0065),"mid":(0.0015,0.005),"macro":(0.0012,0.0042),"mega":(0.001,0.0038)}

    return Baseline(view[t], like[t], comm[t])

def lognorm(sigma: float) -> float:
    return float(np.random.lognormal(0.0, sigma))

def clamp_int(x: float, lo: int = 0, hi: int | None = None) -> int:
    v = int(round(x))
    if hi is not None:
        v = min(v, hi)
    return max(lo, v)


In [0]:
# CÉLULA X — helper para definir data de postagem

from datetime import date, datetime

def resolve_post_date(tipo_data: str, data_custom: str | None = None):
    """
    tipo_data:
      - 'today'  -> usa data atual
      - 'custom' -> usa data_custom (yyyy-mm-dd)
    """
    if tipo_data == "today":
        return date.today()
    
    if tipo_data == "custom":
        if not data_custom:
            raise ValueError("data_custom é obrigatória quando tipo_data='custom'")
        return datetime.strptime(data_custom, "%Y-%m-%d").date()

    raise ValueError("tipo_data inválido (use 'today' ou 'custom')")


In [0]:
# CÉLULA — baseline de métricas por rede social

base = baseline_for(platform, subs)


In [0]:
# CÉLULA 8 — gerar 1 post por execução (SEM random_date)

def generate_post(post_id: int):

    post_date = resolve_post_date(tipo_data, data_custom)

    title = random.choice(VIDEO_TITLES_POOL)
    category = random.choice(CATEGORIES)
    duration_sec = random.randint(12, 55)

    view_factor = random.uniform(*base.view_factor_range)
    volatility = lognorm(0.45)

    viral_boost = 1.0
    if random.random() < 0.06:
        viral_boost = random.uniform(1.8, 6.0)

    expected_views = subs * view_factor * volatility * viral_boost
    hard_cap = max(2_000, int(subs * 35))
    views = clamp_int(expected_views, 0, hard_cap)

    like_rate = random.uniform(*base.like_rate_range)
    comm_rate = random.uniform(*base.comment_rate_range)

    likes = clamp_int(views * like_rate * lognorm(0.2))
    comments = clamp_int(views * comm_rate * lognorm(0.25))

    posted_at = datetime.combine(
        post_date,
        datetime.min.time()
    ) + timedelta(
        hours=random.choice([9, 12, 18, 20, 21, 22]),
        minutes=random.randint(0, 59),
    )

    return {
        "faculdade": faculdade,
        "qtd_inscritos": subs,
        "rede_social": platform,
        "data_postagem": post_date.isoformat(),
        "posted_at": posted_at.isoformat(timespec="seconds"),
        "video_id": str(uuid.uuid4()),
        "titulo_video": title,
        "categoria": category,
        "duracao_seg": duration_sec,
        "views": views,
        "likes": likes,
        "comentarios": comments,

        # métricas de marketing (para Gold)
        "engagement_rate": round((likes + comments) / views, 4) if views > 0 else 0,
        "leads_estimados": clamp_int(views * random.uniform(0.002, 0.015)),
        "ctr_estimado": round(random.uniform(0.6, 2.5), 2),

        "run_id": run_id,
        "ingestion_ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    }


In [0]:
# CÉLULA — path de saída (bronze)

# ajuste se quiser outro bucket/base
out_dir = "s3://capstone-impacta/Capstone/bronze"

dbutils.fs.mkdirs(out_dir)


In [0]:
post = generate_post(post_id=1)

df = spark.createDataFrame([post])

(
    df
    .write
    .mode("append")
    .partitionBy("rede_social", "data_postagem")
    .json(out_dir)
)

#display(df)
